I need Chemical Formula, Cell Parameters, Space Group

In [5]:
from pymatgen.io.cif import CifParser

# Specify the path to your CIF file
cif_path = "ICSD_CollCode257256.cif"

# Create a CIF parser and parse the CIF file
parser = CifParser(cif_path)
structure = parser.parse_structures(primitive=True)[0]

# Print information about the crystal structure
print("Lattice Parameters:")
print("a =", structure.lattice.a)
print("b =", structure.lattice.b)
print("c =", structure.lattice.c)
print()

print("Space Group:", structure.get_space_group_info())
print()

print("Atomic Positions:")
for site in structure.sites:
    print(f"{site.specie}: {site.frac_coords}")


Lattice Parameters:
a = 4.0848
b = 4.0848
c = 4.0848

Space Group: ('Pm-3m', 221)

Atomic Positions:
Cs+: [0. 0. 0.]
Cl-: [0.5 0.5 0.5]


In [7]:
import numpy as np
coordinates_array = np.empty((0, 3), dtype=float)

# Loop through atomic sites and append coordinates to the array
for site in structure.sites:
    coordinates_array = np.vstack([coordinates_array, site.frac_coords])

# Print the resulting NumPy array
print("Atomic Coordinates Array:")
print(coordinates_array)

cartesian_coordinates = coordinates_array * np.array([structure.lattice.a, structure.lattice.b, structure.lattice.c])
print("Cartesian Coordinates Array:")
print(cartesian_coordinates)

Atomic Coordinates Array:
[[0.  0.  0. ]
 [0.5 0.5 0.5]]
Cartesian Coordinates Array:
[[0.     0.     0.    ]
 [2.0424 2.0424 2.0424]]


In [9]:
import plotly.graph_objects as go
# Create a 3D scatter plot
atomic_symbols = [site.specie.symbol for site in structure.sites]
coordinates_array = np.array([site.frac_coords for site in structure.sites])

# Create a 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=coordinates_array[:, 0],
    y=coordinates_array[:, 1],
    z=coordinates_array[:, 2],
    mode='markers',
    marker=dict(
        size=8,
        color='red',  # You can customize the color
        opacity=0.8
    ),
    text=atomic_symbols,
)])

# Set plot layout
fig.update_layout(
    scene=dict(
        aspectmode='cube',
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z'),
    ),
    title='Crystal Structure Visualization',
)

# Show the plot
fig.show()

In [12]:
import numpy as np
from pymatgen.io.cif import CifParser

# Specify the path to your CIF file
cif_path = "ICSD_CollCode257256.cif"

# Create a CIF parser and parse the CIF file
parser = CifParser(cif_path)
structure = parser.parse_structures(primitive=True)[0]

# Extract atomic positions
atomic_symbols = [site.specie.symbol for site in structure.sites]
coordinates_array = np.array([site.frac_coords for site in structure.sites])

# Create a 2x2x2 supercell
supercell_matrix = np.array([[3, 0, 0], [0, 3, 0], [0, 0, 3]])
supercell_coordinates = np.dot(coordinates_array, supercell_matrix)

# Create a 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=supercell_coordinates[:, 0],
    y=supercell_coordinates[:, 1],
    z=supercell_coordinates[:, 2],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        opacity=0.8
    ),
    text=atomic_symbols * 8,  # Repeat atomic symbols for each atom in the supercell
)])

# Set plot layout
fig.update_layout(
    scene=dict(
        aspectmode='cube',
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Z'),
    ),
    title='Crystal Structure Visualization (Supercell)',
)

# Show the plot
fig.show()
